In [65]:
import numpy as np
from PIL import Image,ImageOps
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [66]:

def convertImageToArrayRGB(imagefile, grayscale):
    """
    Input: Image .JPG file
    Output: 2D array representing the image.
    """
    imageObject = Image.open(imagefile)
    if not grayscale :
        return np.array(imageObject)
    else :
        gray_image = ImageOps.grayscale(imageObject)
        return np.array(gray_image)
    

img_2 = convertImageToArrayRGB('L41.jpg', False)
img_2_Gray = convertImageToArrayRGB('L41.jpg', True)
img_2_Gray_4k = convertImageToArrayRGB('L4.jpg', True)
img_2_Gray


array([[26, 27, 27, ..., 26, 26, 26],
       [26, 27, 27, ..., 26, 26, 26],
       [27, 27, 27, ..., 26, 26, 26],
       ...,
       [37, 38, 38, ..., 44, 44, 44],
       [36, 37, 37, ..., 41, 40, 43],
       [34, 34, 35, ..., 38, 38, 39]], dtype=uint8)

In [67]:
def CalculateIntegral(arrayImage):
    """
    Input: 2D array representing the image (feel free to use a predefined function to transform an image into an array).
    Output: 2D array representing the integral image.
    """
    height = len(arrayImage)
    width = len(arrayImage[0])
    
    #Init a 3d array of length, width and RGB values
    si = np.zeros(arrayImage.shape)
    si.astype(int)
    si.setflags(write=1)

    #Add First Coloumn to SI
    for i in range(0,height):
        si[i][0] =  arrayImage[i][0]

    #Get SI    
    for i in range(0,height):
        for j in range(1,width):
            si[i][j] = arrayImage[i][j] + si[i][j-1]
    #Get II
    for j in range(0,width):
        for i in range(1,height):
            si[i][j] = si[i][j] + si[i-1][j]       
    
    return si

In [68]:
#si = CalculateIntegral(img_2)
si_g = CalculateIntegral(img_2_Gray)
si_g_4k = CalculateIntegral(img_2_Gray_4k)
print(si_g[100][100])

693137.0


In [80]:
def CalculateLocalSum(intergralImage, left_corner, right_corner):
    """
    Input: An integral image, and a two pairs of coordinates (𝑝0=(𝑥0,𝑦0),𝑝1=(𝑥1,𝑦1)).
    Output: The local sum for the rectangular area defined by the pair of points (as 𝑝0 being the upper left corner, and 𝑝1 being the lower right corner of this                  rectangular area).
    """
    # Returns Intensity Relative to Origin
    if(left_corner==0 and right_corner!=-1):
        if right_corner[0] == 0 or right_corner[1] == 0:
            #print("BEDANI",intergralImage[right_corner[1]][right_corner[0]],intergralImage[0][0])
            return int(intergralImage[right_corner[1]][right_corner[0]] - intergralImage[0][0])
        return int(intergralImage[right_corner[1]][right_corner[0]] - intergralImage[1][right_corner[0]] - intergralImage[right_corner[1]][1] + intergralImage[1][1])
    #Returns Difference between right and left indicies.
    else:
        #hnafge zero to one
        #print(left_corner,right_corner)
        return int(intergralImage[right_corner[1]][right_corner[0]] + intergralImage[left_corner[1]][left_corner[0]] - intergralImage[left_corner[1]][right_corner[0]]  - intergralImage[right_corner[1]][left_corner[0]] )
        


##print(CalculateLocalSum(si,(300,300),(500,500)))
#test1 = CalculateLocalSum(si_g,(300,300),(500,500))
#test2 = si_g[500][500]+si_g[300][300]-si_g[300][500]-si_g[500][300]
#print(test1==test2)
#print(test1)
#test3 = CalculateLocalSum(si_g,(300,300),(301,301))
#print(test3)

In [81]:
#print(CalculateLocalSum(si_g,0,(500,500)))
#array[height][width]

def EdgeDetect(integral_image, kernel_size,image):
    if(kernel_size%2 ==0 or kernel_size ==1):
        print("Size has to be an Odd Number >1")
        return

    print("Integral Image: \n",integral_image)
    width = len(integral_image[0])
    height = len(integral_image)
    print(width,height)



    
    localSumMatrix = np.copy(integral_image)
    stepSize = math.floor(kernel_size/2)
    h1_result = np.copy(localSumMatrix) 
    h2_result = np.copy(localSumMatrix) 
    magn = np.copy(localSumMatrix) 
    laplace_result = np.copy(localSumMatrix)

    print("StepSize",stepSize)
   # for j in range(0,height):
   #     for i in range(0,width):
   #         localSumMatrix[j][i]= CalculateLocalSum(integral_image,0,(i,j))
    
    
    for j in range(kernel_size,height-kernel_size):
        for i in range(kernel_size,width-kernel_size):
                   #print("pixel @ ",i,j)
                   X1 = int(i - stepSize) #-2,-2
                   Y1 = int(j - stepSize)
                   
                   X2 = int(i + stepSize)
                   Y2 = int(j - stepSize + 1)
                   
                   
                   #print(X1,Y2,X2,Y2)
                   h1_upper = CalculateLocalSum(integral_image,(X1,Y1),(X2,Y2))
                   #print("H1 ",h1_upper)

                   X1 = int(i - stepSize) #-2,-2
                   Y1 = int(j + stepSize -1)
                   
                   X2 = int(i + stepSize)
                   Y2 = int(j + stepSize)
                   h1_lower = CalculateLocalSum(integral_image,(X1,Y1),(X2,Y2))
                   #print("H2 " ,h1_lower)
                   print("H1",h1_lower,h1_upper)
                   h1_all = (h1_upper +(h1_lower*-1)) #check if we need to divide
                   h1_result[j][i] = int(h1_all)

                   X1 = int(i - stepSize)
                   Y1 = int(j - stepSize)

                   X2 = int(i - stepSize + 1)
                   Y2 = int(j + stepSize)
                   
                   h2_left = CalculateLocalSum(integral_image,(X1,Y1),(X2,Y2))

                   X1 = int(i + stepSize -1)
                   Y1 = int(j - stepSize)

                   X2 = int(i + stepSize)
                   Y2 = int(j + stepSize)

                   h2_right = CalculateLocalSum(integral_image,(X1,Y1),(X2,Y2))
                   h2_all = (h2_right +(h2_left*-1)) #check if we need to divide
                   h2_result[j][i] = int(h2_all)
                   #print(h2_all)
                   magn[j][i] = int(math.sqrt((h2_result[j][i])**2 + (h1_result[j][i])**2))
                   if magn[j][i]>=255:
                        magn[j][i] = 255
                   

                   X1 = int(i - stepSize)
                   Y1 = int(j - stepSize)

                   X2 = int(i + stepSize)
                   Y2 = int(j + stepSize)
                   #256*(image-pmin)/(pmax-pmin)
                   laplace_result[j][i]  = abs(CalculateLocalSum(integral_image,(X1,Y1),(X2,Y2)) + image[j][i] + kernel_size**2 - 1)

    
    
    laplace_result = laplace_result  * (np.min(laplace_result)/np.max(laplace_result)) *256

    print("Local Sum Matrix: \n",localSumMatrix)
    print("Magnitude: \n",magn) 
    print("Avrg of Laplace: ",np.mean(laplace_result)) 
    print(localSumMatrix==integral_image)
    image = Image.fromarray(laplace_result)
    image.show()
    
    return #(h1,h2),(h1_abs,h2_abs)

   

In [82]:
test7 = CalculateIntegral(np.arange(501**2).reshape(501,501))
test8 = CalculateLocalSum(si_g,(196,127),(198,127))
print(test8)
#EdgeDetect(test7,5)
EdgeDetect(si_g,5,img_2_Gray)  

13 483
H1 494 464
H1 469 440
H1 445 415
H1 420 390
H1 394 366
H1 367 342
H1 341 320
H1 317 300
H1 292 281
H1 261 265
H1 231 251
H1 202 239
H1 175 228
H1 161 218
H1 151 210
H1 141 203
H1 133 197
H1 122 192
H1 110 187
H1 107 183
H1 116 180
H1 129 179
H1 142 177
H1 146 174
H1 129 170
H1 112 164
H1 95 160
H1 80 155
H1 77 151
H1 76 149
H1 74 145
H1 71 143
H1 682 116
H1 682 118
H1 680 122
H1 679 127
H1 677 131
H1 674 134
H1 673 135
H1 671 137
H1 671 139
H1 671 141
H1 671 144
H1 671 148
H1 670 151
H1 669 154
H1 668 153
H1 669 146
H1 670 139
H1 671 138
H1 673 141
H1 674 150
H1 675 160
H1 676 164
H1 676 170
H1 677 177
H1 678 183
H1 680 196
H1 679 214
H1 655 233
H1 581 257
H1 497 303
H1 425 319
H1 386 328
H1 376 336
H1 363 314
H1 345 313
H1 366 315
H1 359 323
H1 306 330
H1 281 331
H1 207 300
H1 228 276
H1 352 224
H1 450 167
H1 552 144
H1 615 108
H1 637 109
H1 713 136
H1 811 175
H1 887 227
H1 815 267
H1 618 292
H1 456 309
H1 296 322
H1 355 344
H1 578 374
H1 740 426
H1 915 498
H1 1015 596
H1 1016 

In [ ]:
type()